```latex
\makeatletter
\edef\@figdir{_figurecache}
\edef\@pgfrootdir{figures}
\newcommand\setpgfroot[1]{\def\@pgfrootdir{#1}}
\let\oldpgfimage\pgfimage
\renewcommand{\pgfimage}[2][]{\oldpgfimage[#1]{\@pgfrootdir/#2}}

\newcommand{\@pgfpreamble}[1]{%
	\unexpanded{%
		\documentclass[class=dionsthesis]{standalone}^^J
		\setpgfroot{#1}%
	}%
}

\newcounter{@pgfcounter}
\newwrite\@pgfout
\newread\@pgfin

\newcommand{\importpgf}[3][]{%
	\IfFileExists{#2/#3}{}{\ClassError{dionsthesis}{importpgf: File not found}{File #2/#3 not found}}%
	\edef\@figfile{\jobname-\the@pgfcounter}%
	\providecommand{\@writetempfile}{}%
	\renewcommand{\@writetempfile}[1]%
	{%
		\immediate\openout\@pgfout=##1%
		\immediate\write\@pgfout{\@pgfpreamble{#2}}%
		\immediate\write\@pgfout{\string\\begin{document}}%
		\immediate\openin\@pgfin=#2/#3%
		\begingroup\endlinechar=-1%
			\loop\unless\ifeof\@pgfin%
				\readline\@pgfin to \@fileline%
				\ifx\@fileline\@empty\else%
					\immediate\write\@pgfout{\@fileline}%
				\fi%
			\repeat%
		\endgroup%
		\immediate\closein\@pgfin%
		\immediate\write\@pgfout{\string\end{document}}%
		\immediate\closeout\@pgfout%
	}%
	\def\@compile%
	{%
		\immediate\write18{pdflatex -interaction=batchmode -output-directory="\@figdir" \@figdir/\@figfile.tex}%
	}%
	\IfFileExists{\@figdir/\@figfile.pdf}%
	{%
		\@writetempfile{\@figdir/tmp.tex}%
		\edef\@hashold{\pdfmdfivesum file {\@figdir/\@figfile.tex}}%
		\edef\@hashnew{\pdfmdfivesum file {\@figdir/tmp.tex}}%
		\ifnum\pdfstrcmp{\@hashold}{\@hashnew}=0%
			\relax%
		\else%
			\@writetempfile{\@figdir/\@figfile.tex}%
			\@compile%
		\fi%
	}%
	{%
		\@writetempfile{\@figdir/\@figfile.tex}%
		\@compile%
	}%
	\IfFileExists{\@figdir/\@figfile.pdf}%
	{\includegraphics[#1]{\@figdir/\@figfile.pdf}}%
	{\errmessage{Error during compilation of figure #2/#3}}%
	\stepcounter{@pgfcounter}%
}
\makeatother
```

You can use it like so:

```latex
\begin{figure}
    \importpgf{path/to/figure}{figure.pgf}
\end{figure}
```

and make sure to compile your file with the flag `-shell-escape` (otherwise, `pdflatex` cannot be called to compile your figures).

Note that I'm not exactly a LaTeX-pro, so I can't give any guarantee that this works in any situation, or that the implementation is optimal. Also note that this doesn't detect whether your figures were compiled successfully, so if you get a cryptic error message or the comiler hangs, make sure that your figures compile properly, and delete the `_figurecache` folder.

### How does it work?

When encountering the command `\importpgf`, LaTeX looks up the given file, and writes its contents, along with the preamble from the command `\@pgfpreamble`, into a temporary file. The command calculates the MD5 checksum of this temporary file and, if it exists, of a corresponding `.tex` file for the given figure that was written during a previous compilation using `\importpgf`. If the hash matches, the command just returns the PDF file corresponding to the `.tex` file with `\includegraphics`. Otherwise, the `.tex` file is overwritten with the temporary file, compiled to a PDF, and then returned with `\includegraphics`. Because of that, the first compilation may take a lot of time (all figures are compiled to PDF one-by-one), but the following compilations should be fast if you don't change any of the figures.

The code worked fine on my setup (Ubuntu 16.04 with TexLive), but I didn't test it on any non-Linux machines, so your mileage may vary. If you run into problems, feel free to contact me.